# 6 GLMの応用範囲を広げる -ロジスティック回帰など-

In [1]:
using CSV
using DataFrames
using Distributions
using GLM
using LaTeXStrings
using Plots
using StatsBase

## 6.2 例題 : 上限のあるカウントデータ

In [2]:
df = CSV.read("data4a.csv");

In [3]:
df

,N,y,x,f
,Int64⍰,Int64⍰,Float64⍰,String⍰
1,8,1,9.76,C
2,8,6,10.48,C
3,8,5,10.83,C
4,8,6,10.94,C
5,8,1,9.37,C
6,8,1,8.81,C
7,8,3,9.49,C
8,8,6,11.02,C
9,8,0,7.97,C


In [ ]:
plot(df.x, df.y, group=df.f, seriestype=:scatter, xlabel="Plant size", ylabel="Number of seeds")

In [ ]:
describe(df)

## 6.4 ロジスティック回帰とロジットリンク関数

In [ ]:
function logistic(z)
    1 / (1 + exp(-z))
end

In [ ]:
z = -6:0.1:6
plot(z, logistic.(z), title="Logistic function", xlabel="Linear predictor", ylabel="Probability")

### 6.4.2 パラメータ推定
Juliaでは、Binomial分布の場合、応答変数はfractionにしなければならない  
https://github.com/JuliaStats/GLM.jl/issues/228#issuecomment-387340111

In [ ]:
df.yy = df.y ./ df.N
result = glm(@formula(yy ~ x + f), df, Binomial(), wts=fill(8.0, nrow(df)))

In [ ]:
dfc = df[df.f .== "C", :];
plot(dfc.x, dfc.y, seriestype=:scatter, label="C")
xx = DataFrame(x=range(minimum(dfc.x), maximum(dfc.x), length=100), f="C")
yy = predict(result, xx) * 8.0
plot!(xx.x, yy)

In [ ]:
dft = df[df.f .== "T", :];
plot(dft.x, dft.y, seriestype=:scatter, label="T", color=:red)
xx = DataFrame(x=range(minimum(dft.x), maximum(dft.x), length=100), f="T")
yy = predict(result, xx) * 8.0
plot!(xx.x, yy)

In [ ]:
typeof(result)

### 6.4.4 ロジスティック回帰のモデル選択
$k, \log L^*, \mbox{deriance} - 2\log L^*, \mbox{residual deviance}, \mbox{AIC}$

In [ ]:
function model_selection_table(result)
    dof(result), loglikelihood(result), -2loglikelihood(result), deviance(const_model), aic(result)
end

In [ ]:
const_model = glm(@formula(yy ~ 1), df, Binomial(), wts=fill(8.0, nrow(df)))
model_selection_table(const_model)

In [ ]:
f_model = glm(@formula(yy ~ 1 + f), df, Binomial(), wts=fill(8.0, nrow(df)))
model_selection_table(f_model)

In [ ]:
x_model = glm(@formula(yy ~ 1 + x), df, Binomial(), wts=fill(8.0, nrow(df)))
model_selection_table(x_model)

In [ ]:
xf_model = glm(@formula(yy ~ 1 + x + f), df, Binomial(), wts=fill(8.0, nrow(df)))
model_selection_table(xf_model)

## 6.5 交互作用項の入った線形予測子

In [ ]:
interaction_model = glm(@formula(yy ~ x + f + x * f), df, Binomial(), wts=fill(8.0, nrow(df)))

In [ ]:
model_selection_table(interaction_model)

## 6.6 割算値の統計モデリングはやめよう
### 6.6.1 割算値いらずのオフセット項わざ

In [ ]:
df_population = CSV.read("data4b.csv")

In [ ]:
plot(df_population.A, df_population.y, seriestype=:scatter, 
    markeralpha=df_population.x, label="",
    xlabel="Area", ylabel="Plant population")

In [ ]:
population_reseult = glm(@formula(y ~ x), df_population, Poisson(), offset=log.(df_population.A))

### 明るさごとの平均個体数の予測

In [ ]:
plt = plot(df_population.A, df_population.y, seriestype=:scatter, 
    markeralpha=df_population.x, label="", 
    title="Prediction",
    xlabel="Area", ylabel="Plant population",
    legendtitle = "Brightness", legend = :topleft)
for j = 0.1:0.2:0.9
    xx = DataFrame(A=range(minimum(df_population.A), maximum(df_population.A), length=100), x=j)
    yy = predict(population_reseult, xx, offset=log.(xx.A))
    plot!(xx.A, yy, lw=3, color=:red, linealpha=j, label=j)
end
display(plt)

## 6.7 正規分布とその尤度

In [ ]:
y = -5:0.1:5
plot(y, pdf.(Normal(), y), label="", 
    title=L"\mu=0, \sigma=1",
    ylabel="Probability density")

In [ ]:
plot(y, pdf.(Normal(0, 3), y), label="", 
    title=L"\mu=0, \sigma=3",
    ylabel="Probability d3nsity")

In [ ]:
plot(y, pdf.(Normal(2, 1), y), label="", 
    title=L"\mu=2, \sigma=1",
    ylabel="Probability d3nsity")

$p(1.2 \le y \le 1.8 | \mu, \sigma)$を評価する

In [ ]:
cdf(Normal(), 1.8) - cdf(Normal(), 1.2)

近似

In [ ]:
pdf.(Normal(), 1.5) * 0.6

## 6.8 ガンマ分布のGLM

In [ ]:
d = CSV.read("d.csv");

In [ ]:
d.logx = log.(d.x)
d

In [ ]:
glm(@formula(y ~ logx), d, Gamma(), LogLink())